<a href="https://colab.research.google.com/github/shineloveyc/Doing_ML/blob/master/Chapter8_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install pyprind

In [0]:
#load the library
import pandas as pd
import tarfile
from pathlib import Path
import pyprind #python progress indicator
import os
from fastai.vision import *

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'Colab Notebooks/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#set up the path to read data
path = Path(base_dir + 'data/')
print(path)

/content/gdrive/My Drive/Colab Notebooks/data


In [0]:
'''get review data set IMDB movie review data
tar = tarfile.open(path/'aclImdb_v1.tar.gz')
tar.extractall()'''

"get review data set IMDB movie review data\ntar = tarfile.open(path/'aclImdb_v1.tar.gz')\ntar.extractall()"

In [0]:
#check current working directory
os.chdir(path)

In [0]:
'''
basepath = 'aclImdb'

#The PyPrind (Python Progress Indicator) module provides a progress bar and a percentage indicator object 
#that let you track the progress of a loop structure or other iterative computation.

labels = {'pos':1, 'neg':'0'}
pbar = pyprind.ProgBar(50000)

df = pd.DataFrame()
#iterate over the train and test subdirectories in the main aclImdb directory and read individual text files from the pos and neg subdirectories
#and eventually append to the df, together with class label 1 pos, 0 neg
for s in ('test', 'train'):
  for l in ('pos', 'neg'):
    path = os.path.join(basepath, s, l)
    for file in os.listdir(path):
      with open(os.path.join(path, file),
          'r', encoding = 'utf-8') as infile:
        txt = infile.read()
      df = df.append([[txt, labels[l]]],
                     ignore_index = True)
      pbar.update()
df.columns = ['review', 'sentiment']

'''

"\nbasepath = 'aclImdb'\n\n#The PyPrind (Python Progress Indicator) module provides a progress bar and a percentage indicator object \n#that let you track the progress of a loop structure or other iterative computation.\n\nlabels = {'pos':1, 'neg':'0'}\npbar = pyprind.ProgBar(50000)\n\ndf = pd.DataFrame()\n#iterate over the train and test subdirectories in the main aclImdb directory and read individual text files from the pos and neg subdirectories\n#and eventually append to the df, together with class label 1 pos, 0 neg\nfor s in ('test', 'train'):\n  for l in ('pos', 'neg'):\n    path = os.path.join(basepath, s, l)\n    for file in os.listdir(path):\n      with open(os.path.join(path, file),\n          'r', encoding = 'utf-8') as infile:\n        txt = infile.read()\n      df = df.append([[txt, labels[l]]],\n                     ignore_index = True)\n      pbar.update()\ndf.columns = ['review', 'sentiment']\n\n"

In [0]:
'''
import numpy as np

#makes the random numbers predictable
np.random.seed(0)

#Randomly permute a sequence, or return a permuted range.
df = df.reindex(np.random.permutation(df.index))

#store all shuffled and assembled data to csv file
df.to_csv('movie_data.csv', index = False, encoding = 'utf-8')'''

"\nimport numpy as np\n\n#makes the random numbers predictable\nnp.random.seed(0)\n\n#Randomly permute a sequence, or return a permuted range.\ndf = df.reindex(np.random.permutation(df.index))\n\n#store all shuffled and assembled data to csv file\ndf.to_csv('movie_data.csv', index = False, encoding = 'utf-8')"

In [0]:
df = pd.read_csv('movie_data.csv', encoding = 'utf-8')
df.head(3)

,review,sentiment
0,I just finished watching this movie and I must...,1
1,Dustin Hoffman's debut feature isn't as bad as...,0
2,"One of the best ""Amitabh comeback"" movies I li...",1


## Bag of words model

In [0]:
#demo countvectorizer 
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array(['The sun is shining',
                 'The weather is sweet',
                 'The sun is shining, the weather is sweet, and one and one is two'])

#turn the document object to numeric vectors
bg = count.fit_transform(docs)

In [0]:
#check the content of vocabulary, a dictionary store the word
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [0]:
#check the array object
print(bg.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [0]:
#deploy tfidf
from sklearn.feature_extraction.text import TfidfTransformer

The L2 norm calculates the distance of the vector coordinate from the origin of the vector space. As such, it is also known as the Euclidean norm as it is calculated as the Euclidean distance from the origin. The result is a positive distance value.

In [0]:
'''Smooth idf weights by adding one to document frequencies, as if an extra document was seen containing every term 
in the collection exactly once. Prevents zero divisions.'''
tfidf = TfidfTransformer(use_idf=True,norm = 'l2',smooth_idf = True)

np.set_printoptions(precision=2)

print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


## Clean IMDB text data

In [0]:
#dispaly last 50 char of the first document
df.loc[0,'review'][-50:]

'ee bit sad. And not necessarily so. Cheers Furdion'

In [0]:
#remove HTML markup and punctuation
import re # using regular expression clean the data
def preprocessor(text):
  text = re.sub('<[^>]*>', '', text) #remove HTML markup
  emotions = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)#store emotions
  text = (re.sub('[\W]+', ' ', text.lower()) +' '.join(emotions).replace('-', '')) #remove all non-word char and covert text to lowercase cha

  return text

In [0]:
print(preprocessor(df.loc[0, 'review'][-50:]))

str_test = '</a>This :) is :( a test :-)!'
preprocessor(str_test)

ee bit sad and not necessarily so cheers furdion


'this is a test :) :( :)'

In [0]:
#apply preprocess fun to the df

df['review'] = df['review'].apply(preprocessor)

In [0]:
#precessing docs to tokens
def tokenizer(text):
  return text.split()

tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [0]:
#word stemming by using Porter stemming NLTK package
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):
  return [porter.stem(word) for word in text.split()]

tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [0]:
import nltk

#download stopword
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
#load and apply the English stop-word 
from nltk.corpus import stopwords

stop = stopwords.words('english')

[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

## Document classification

In [0]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values

X_test  = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import  TfidfVectorizer

In [0]:
#combine countvectorizer and TfidfTransformer object
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase = False,
                        preprocessor = None)

In [0]:
#create grid search with two different paramenters

#first parameter use default settings (use_idf=True, smooth_idf=True, and norm='l2') to calculate the tf-idfs;
param_grid = [{'vect__ngram_range': [(1,1)],
              'vect__stop_words': [stop, None],
              'vect__tokenizer': [tokenizer,
                                  tokenizer_porter],
              'clf__penalty': ['l2'],
'clf__C': [1.0, 10.0, 100.0]}, {'vect__ngram_range': [(1,1)],
              'vect__stop_words': [stop, None],
              'vect__tokenizer': [tokenizer,
                                  tokenizer_porter],
              'vect__use_idf':[False],
'vect__norm':[None], 'clf__penalty': ['l2'],'clf__C': [1.0, 10.0, 100.0]}
]

In [0]:
#build the pipeline
lr_tfidf = Pipeline([('vect', tfidf), #add vectorization object name and object to pipeline
                     ('clf', LogisticRegression(random_state=0))]) #add lg object and name

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid=param_grid,
                           scoring = 'accuracy',
                           cv = 5, verbose = 1, n_jobs =1)

In [0]:
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/usr/local/lib/python3.6/dist-packages/

KeyboardInterrupt: ignored

In [0]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)

by using the regular tokenizer without Porter stemming, no stop-word library, and tf-idfs in combination with a logistic regression classifier that uses L2-regularization with the regularization strength C of 10.0.

In [0]:
#print the avg cv accuracy score
print('CV Accuracy: %3.f' % gs_lr_tgidf.best_score_)

In [0]:
#print test accuracy
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

#Web Training

In [0]:
#use loaded object to preprocess document
label = {0:'negative', 1:'positive'}

example = ['I love this movie']

X = vect.transform(example)

print("predication : %s\n Probability: %.2f%%" %\(label[clf.predict(X)[0]],
                                                  np.max(clf.predict_proba(X))*100))